# Xtreme Gradient Booster Model

#### Goal of This File:

##### 1. Import Libraries, Helper Functions, and Constants ==> Data Sourcing and Munging ==> Utility Functions ==> Loading the Data ==> Merging all Individual Files

##### 2. Imbalanced Data Solution ==> Hyperparameters

##### 3. XGB

##### 4. Conclusion ==> Results

## 1. Import Libraries, Helper Functions and Load Constants

In [1]:
!pip install pandas numpy nltk scikit-learn wordcloud\
seaborn gensim tensorflow imblearn xgboost matplotlib unrar pyunpack more-itertools patool transformers > /dev/null

The system cannot find the path specified.


In [2]:
VAST = True

if VAST:
    !sudo apt-get install unrar
    !sudo apt-get install rar
    
GDRIVE = False

'sudo' is not recognized as an internal or external command,
operable program or batch file.
'sudo' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
UTILS_URL = 'https://f005.backblazeb2.com/file/gp-support-files/nlp_ai_utils.py'
UPDATING_VALUES_URL = 'https://f005.backblazeb2.com/file/gp-support-files/updating_values.py'
ALL_LIBS_URL = 'https://f005.backblazeb2.com/file/gp-support-files/all_libs_dl.py'
CHUNKS_URLS_FILE_URL = 'https://f005.backblazeb2.com/file/gp-support-files/chunks_urls.py'

UTILS_FILE_NAME = 'nlp_ai_utils'
UPDATING_VALUES_FILE_NAME = 'updating_values'
ALL_LIBS_FILE_NAME = 'all_libs_dl'
CHUNKS_URLS_FILE_NAME = 'chunks_urls'

DEP_FILE_EXT = '.py'

In [4]:
import requests
import os

def get_dependencies(url,file_name,file_extension):
    if os.path.exists(file_name + file_extension):
        return print(file_name + " already exists.")
    else:
        print(f"downloading {file_name}...")
        r = requests.get(url)
        with open(file_name + file_extension, 'wb') as fd:
            for chunk in r.iter_content():
                fd.write(chunk)

In [5]:
get_dependencies(UTILS_URL,UTILS_FILE_NAME,DEP_FILE_EXT)
get_dependencies(UPDATING_VALUES_URL,UPDATING_VALUES_FILE_NAME,DEP_FILE_EXT)
get_dependencies(ALL_LIBS_URL,ALL_LIBS_FILE_NAME,DEP_FILE_EXT)
get_dependencies(CHUNKS_URLS_FILE_URL,CHUNKS_URLS_FILE_NAME,DEP_FILE_EXT)

nlp_ai_utils already exists.
updating_values already exists.
all_libs_dl already exists.
chunks_urls already exists.


In [6]:
from nlp_ai_utils import *
from chunks_urls import CHUNKS_URLS
from updating_values import *

c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already u

In [7]:
TF_ENABLE_ONEDNN_OPTS = 0
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

LIMIT = 6
DATA_URLS = CHUNKS_URLS[:LIMIT]
GLOVE_URL = 'https://f005.backblazeb2.com/file/glove-embeddings-dims/glove.6B.100d.txt'
READY_DATA_URL = 'https://f005.backblazeb2.com/file/yelp-dataset-ready-for-models/ready_for_models.rar'
CLASS_WEIGHTS_URL = 'https://f005.backblazeb2.com/file/yelp-dataset-ready-for-models/class_weights.pickle'
UNIQUE_WORDS_URL = 'https://f005.backblazeb2.com/file/yelp-dataset-ready-for-models/unique_words.pickle'

if GDRIVE:
    PICKLES_DIR = "pickle_files"
    READY_DATASET_PATH = "larger_dataset/ready_for_models/"
    LARGER_DATASET_PATH = "larger_dataset"
    GLOVE_FILES_DIR = "glove_files"
    SAVED_MODELS_DIR = 'saved_models'
else:
    PICKLES_DIR = "../pickle_files"    
    LARGER_DATASET_PATH = "../larger_dataset"
    READY_DATASET_PATH = "../larger_dataset/ready_for_models/"
    GLOVE_FILES_DIR = "../glove_files"
    SAVED_MODELS_DIR = '../saved_models'
    
PREPROCESSED_CHUNKS_PATH = LARGER_DATASET_PATH + "/preprocessed_data_chunks/"
BASE_FILE_NAME = "chunk_"
FILE_FORMAT = ".csv"
DATA_IN_CHUNK = 116505
if LIMIT == 60:
    ACTUAL_DATA_SHAPE = DATA_IN_CHUNK * LIMIT + 20
else:
    ACTUAL_DATA_SHAPE = DATA_IN_CHUNK * LIMIT
RANDOM_STATE = CONST_RANDOM_STATE
DELETE_PICKLES_AFTER_TRAINING = True
USE_READY_DATA = False
BALANCE_DATA = True
USE_MODIFIED_LABELS = True
TRAIN_PERCENT = 0.8

In [8]:
from collections import Counter

In [9]:
def get_embedding(text, model, dim):
    text = text.lower().split()
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    
    for i,word in enumerate(words):
            v = model.wv[word]
            vectors[i] = v*(words[word]/total)
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

### 1.1 Data Sourcing and Munging

#### 1.1.1 Loading The Data

In [10]:
if not USE_READY_DATA:
    if not os.path.exists(LARGER_DATASET_PATH):
        os.mkdir(LARGER_DATASET_PATH)
    if not os.path.exists(PREPROCESSED_CHUNKS_PATH):
        os.mkdir(PREPROCESSED_CHUNKS_PATH)
    get_chunks(DATA_URLS,LIMIT,1,BASE_FILE_NAME,PREPROCESSED_CHUNKS_PATH,FILE_FORMAT)

chunk_1 already exists.
chunk_2 already exists.
chunk_3 already exists.
chunk_4 already exists.
chunk_5 already exists.
chunk_6 already exists.


#### 1.1.2 Merging all Individual Files

In [11]:
#get all names of downloaded files
if not USE_READY_DATA:
    all_file_names = get_all_file_names(BASE_FILE_NAME,LIMIT)

In [12]:
#read all chunks into a list
if not USE_READY_DATA:
    list_dfs = read_chunks(all_file_names,PREPROCESSED_CHUNKS_PATH,FILE_FORMAT)

In [13]:
#concatenate all chunks into a singular df
if not USE_READY_DATA:
    df = group_up_chunks(list_dfs)

In [14]:
#check how much of the data was actually downloaded
if not USE_READY_DATA:
    percent_loaded = check_no_missing_data(df.shape[0],ACTUAL_DATA_SHAPE)
    percent_loaded

In [15]:
if not USE_READY_DATA:
    review_data = df[['text', 'stars']].copy()

In [16]:
if not USE_READY_DATA:
    review_data.reset_index(inplace = True)
    review_data.drop(['index'],axis = 1,inplace = True)

In [17]:
if not USE_READY_DATA:
    review_data.rename(columns = {'text':'full_review_text','stars':'star_rating'}, inplace = True)

In [18]:
if not USE_READY_DATA:
    review_data.isnull().sum()

In [19]:
if not USE_READY_DATA:
    review_data.dropna(inplace = True)

In [20]:
if not USE_READY_DATA:
    review_data['full_review_text'].replace('', np.nan, inplace=True)
    review_data.dropna(inplace = True)

## 2. Imbalanced Data Solution

In [21]:
if BALANCE_DATA:
    label_1 = review_data[review_data['star_rating'] == 1]
    label_2 = review_data[review_data['star_rating'] == 2]
    label_3 = review_data[review_data['star_rating'] == 3]
    label_4 = review_data[review_data['star_rating'] == 4]
    label_5 = review_data[review_data['star_rating'] == 5]

    minority_class = min([label_1.shape[0],label_2.shape[0],label_3.shape[0],label_4.shape[0],label_5.shape[0]])

In [22]:
if BALANCE_DATA:
    if USE_MODIFIED_LABELS:
        review_data_label_1 = label_1[:int(minority_class / 2)]
        review_data_label_2 = label_2[:int(minority_class / 2)]
        review_data_label_3 = label_3[:int(minority_class)]
        review_data_label_4 = label_4[:int(minority_class / 2)]
        review_data_label_5 = label_5[:int(minority_class / 2)]

    else:
        review_data_label_1 = label_1[:int(minority_class)]
        review_data_label_2 = label_2[:int(minority_class)]
        review_data_label_3 = label_3[:int(minority_class)]
        review_data_label_4 = label_4[:int(minority_class)]
        review_data_label_5 = label_5[:int(minority_class)]

    review_data = pd.concat([review_data_label_1,review_data_label_2,review_data_label_3,review_data_label_4,review_data_label_5])

    print(review_data.shape)

    print(review_data['star_rating'].value_counts())

(160320, 2)
3.0    53440
1.0    26720
2.0    26720
4.0    26720
5.0    26720
Name: star_rating, dtype: int64


In [23]:
if BALANCE_DATA:
    review_data = review_data.sample(frac=1,random_state=RANDOM_STATE)
    review_data.reset_index(drop=True,inplace=True)

In [24]:
if not USE_READY_DATA:
    X = review_data['full_review_text']
    y = review_data['star_rating']

In [25]:
if not USE_READY_DATA:
    X = pd.Series([str(text) for text in X])

In [26]:
if USE_MODIFIED_LABELS:
    translated_labels = translate_labels(y)
    y = pd.Series(encode_sent(translated_labels))

In [27]:
print(y.value_counts())

2    53440
3    53440
1    53440
dtype: int64


In [28]:
if not USE_READY_DATA:
    y = y - 1

#### Classes' shapes before and after (if USE_MODIFIED_LABELS = True):

BEFORE: 1 ==> Very Bad ==> AFTER: 2 ==> Negative Sentiment

BEFORE: 2 ==> Bad ==> AFTER: 2  ==> Negative Sentiment

BEFORE: 3 ==> Ok ==> AFTER: 3  ==> Neutral Sentiment

BEFORE: 4 ==> Good ==> AFTER: 1  ==> Positive Sentiment

BEFORE: 5 ==> Very Good ==> AFTER: 1  ==> Positive Sentiment

THEN: 1 is subtracted from each label. So, the Labels go FROM: 2,3,1 TO: 1,2,0

#### Classes' shapes before and after (if USE_MODIFIED_LABELS = False):

1 is subtracted from each label. So, the Labels go FROM: 1,2,3,4,5 TO: 0,1,2,3,4

Why is 1 subracted from each label? To bring the data into the preferred shape of the class weights in both sklearn and keras (starting label is 0, not 1).

### Hyperparameters

In [29]:
# VOCAB_SIZE = len(UNIQUE_WORDS)
# LSTM_UNITS = 64
# DENSE_UNITS = 512
# LABELS_COUNT = len(y.unique())
# EMBEDDING_DIM = 100
# MAX_TEXT_LEN = 250
# CONV_FILTERS = 60
# CONV_KERNEL_SIZE = 5
# DROPOUT_VAL = 0.5
# TRUNC_TYPE = 'post'
# PADDING_TYPE = 'post'
# OOV_TOKEN = "<OOV>"
# BATCH_SIZE = 128
# EPOCHS = 40

##### ==> here is a quick explaination of how the dataset will be split using a smaller sample example.
##### ==> dataset => 100
##### ==> train_set => tr_s (example: 80)
##### ==> valid_set => vs (example: 10)
##### ==> test_set => te_s (example: 10)
##### ==> t = tr_s (80) + vs (10)
##### ==> train_set = x[:80]
##### ==> valid_set = x[80:t]
##### ==> test_set = x[t:] why t? because => vs = ts

##### use this guideline if you are confused about how the train-validation-test split was done. Also, this is a future guide for me as well in case I forget.

* train_set_size = 6,990,280 * 0.8 = 5,592,224
* valid_set_size = 6,990,280 * 0.1 = 699,028
* train_plus_valid = 5,592,224 + 699,028 = 6,291,252

==> To Confirm: test_size = 6,990,280 - 6,291,252 = 699,028

* train_set = [0:5,592,224]
* train_labels = [0:5,592,224]
* validation_set = [5,592,224:6,291,252] ==> 699,028
* validation_labels = [5,592,224:6,291,252] ==> 699,028
* test_set = [6,291,252,6,990,280] ==> 699,028
* test_labels = [6,291,252,6,990,280] ==> 699,028

In [30]:
ACTUAL_DATA_SHAPE = len(X)

In [31]:
X.reset_index(drop=True,inplace=True)
y.reset_index(drop=True,inplace=True)

In [32]:
VALID_TEST_PERCENT = round(1 - TRAIN_PERCENT,1) / 2
TRAIN_SIZE = int(ACTUAL_DATA_SHAPE * TRAIN_PERCENT)
VALID_TEST_SIZE = int(ACTUAL_DATA_SHAPE * VALID_TEST_PERCENT)
TOTAL_TEST_SIZE = TRAIN_SIZE + VALID_TEST_SIZE

In [33]:
ACTUAL_DATA_SHAPE = len(X)

if BALANCE_DATA:
    temp = pd.DataFrame({'text':X,'labels':y})
    shuffled = temp.sample(frac=1,random_state=RANDOM_STATE)
    X = shuffled['text']
    y = shuffled['labels']

In [34]:
X_W2V = X

In [35]:
w2v_model = Word2Vec(sentences=[nltk.word_tokenize(text) for text in X_W2V], vector_size=100, window=5, min_count=1, workers=4)

In [36]:
word_index_w2v = {}

for i, word in enumerate(w2v_model.wv.key_to_index):
    word_index_w2v[word] = i

In [37]:
vocab_size = len(word_index_w2v)
embedding_dim_w2v = w2v_model.vector_size

embedding_matrix_w2v = np.zeros((vocab_size, embedding_dim_w2v))

for word, i in word_index_w2v.items():
    if word in w2v_model.wv.key_to_index:
        embedding_matrix_w2v[i] = w2v_model.wv.get_vector(word)

In [38]:
X_text_w2v = np.zeros((X_W2V.shape[0], 100))

for i, text in enumerate(X_W2V.values):
    X_text_w2v[i] = get_embedding(text, w2v_model, 100)

In [39]:
train_set = X_text_w2v[:TRAIN_SIZE]
validation_set = X_text_w2v[TRAIN_SIZE:TOTAL_TEST_SIZE]
test_set = X_text_w2v[TOTAL_TEST_SIZE:]

In [40]:
train_labels = y[:TRAIN_SIZE]
validation_labels = y[TRAIN_SIZE:TOTAL_TEST_SIZE]
test_labels = y[TOTAL_TEST_SIZE:]

In [41]:
w2v_sklearn = CountVectorizer()
X_sklearn = w2v_sklearn.fit_transform(X)

In [42]:
train_set_sklearn = X_sklearn[:TRAIN_SIZE]
validation_set_sklearn = X_sklearn[TRAIN_SIZE:TOTAL_TEST_SIZE]
test_set_sklearn = X_sklearn[TOTAL_TEST_SIZE:]

In [43]:
train_set_sklearn = np.array(train_set_sklearn)

validation_set_sklearn = np.array(validation_set_sklearn)

test_set_sklearn = np.array(test_set_sklearn)

In [44]:
train_labels_sklearn = y[:TRAIN_SIZE]
validation_labels_sklearn = y[TRAIN_SIZE:TOTAL_TEST_SIZE]
test_labels_sklearn = y[TOTAL_TEST_SIZE:]

## 3. XGB

In [45]:
xgb_model_gensim = XGBClassifier(random_state=RANDOM_STATE)

In [46]:
xgb_model_gensim.fit(train_set,train_labels)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [47]:
model_preds_gensim = xgb_model_gensim.predict(validation_set)

In [48]:
xgb_model_sklearn = XGBClassifier(random_state=RANDOM_STATE)

In [49]:
print(f"accuracy of XGB with gensim: {metrics.accuracy_score(validation_labels,model_preds_gensim)}")

accuracy of XGB with gensim: 0.689246506986028


## 4. Conclusion

### 4.1 Results